In [1]:
!pip install transformers


In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

def ensemble_classification(sentences, model_name_1, model_name_2):

    tokenizer_1 = AutoTokenizer.from_pretrained(model_name_1)
    model_1 = AutoModelForSequenceClassification.from_pretrained(model_name_1)

    tokenizer_2 = AutoTokenizer.from_pretrained(model_name_2)
    model_2 = AutoModelForSequenceClassification.from_pretrained(model_name_2)

    final_labels = []


    for text in sentences:

        inputs_1 = tokenizer_1(text, return_tensors="pt")
        inputs_2 = tokenizer_2(text, return_tensors="pt")


        with torch.no_grad():
            outputs_1 = model_1(**inputs_1)
            probs_1 = torch.nn.functional.softmax(outputs_1.logits, dim=-1)


        with torch.no_grad():
            outputs_2 = model_2(**inputs_2)
            probs_2 = torch.nn.functional.softmax(outputs_2.logits, dim=-1)


        avg_probs = (probs_1 + probs_2) / 2


        final_class = avg_probs.argmax().item()

        final_labels.append(final_class)

    return final_labels

sentences = ["новое оруже поступило на базу", "война в Украине", "новости обороны"]
model_name_1 = "bodomerka/Milytary_exp_class_classification"
model_name_2 = "bodomerka/Mil_class_exp_sber_balanssedclass"

labels = ensemble_classification(sentences, model_name_1, model_name_2)

print(labels)


[1, 0, 0]
